# Bias and Fairness Analysis

**Goal**: Evaluate the fairness of the offer completion prediction model across different demographic groups.

**Context from Previous Analyses:**

This analysis builds upon the following completed work:
- **Modeling (03_Modeling.ipynb)**: Random Forest selected as best model (F1: 0.8601, AUC: 0.9277) with 24 features after removing data leakage
- **PCA (04_PCA.ipynb)**: 8 components capture 90% variance with minimal performance loss (1% F1 drop)
- **SHAP (05_SHAP.ipynb)**: Top 3 features (offer_type_discount 21.4%, duration 14.2%, difficulty 9.3%) drive 45% of predictions
- **Segmentation (06_Customer_Segmentation.ipynb)**: 5 customer segments identified with completion rates from 15.7% to 72.6%

**Why Fairness Matters:**
- Avoid discriminatory outcomes in marketing offers
- Ensure equitable customer experience across demographic groups
- Build trust in AI-driven recommendations
- Meet regulatory and ethical standards
- Identify and mitigate bias from modeling decisions

**Protected Attributes Analyzed:**
1. **Gender**: Male, Female, Other, Missing
2. **Age Group**: 18-30, 31-45, 46-60, 61-75, 76+
3. **Income Bracket**: Missing, Low, Medium, High, Very High
4. **Tenure Group**: New, Short-term, Medium-term, Long-term

**Fairness Metrics:**
- **Demographic Parity**: Similar prediction rates across groups (80% rule: 0.8 ≤ ratio ≤ 1.25)
- **Equal Opportunity**: Similar true positive rates across groups
- **Predictive Parity**: Similar precision across groups
- **Disparate Impact**: Ratio of favorable outcomes between groups
- **Overall Accuracy**: Similar accuracy across groups

**Key Questions:**
1. Does the model perform equally well across all demographic groups?
2. Are certain groups systematically over/under-predicted?
3. Do feature importance patterns create unfair advantages for specific groups?
4. Are there intersectional biases (e.g., young low-income customers)?

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, confusion_matrix, roc_auc_score,
                            precision_recall_curve, roc_curve)
import joblib
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Environment ready! ✓")

Environment ready! ✓


## Load Data and Models

In [ ]:
processed_dir = '../Cafe_Rewards_Offers/processed'
models_dir = '../Cafe_Rewards_Offers/models'

X_test = joblib.load(f'{processed_dir}/X_test_scaled.pkl')
y_test = joblib.load(f'{processed_dir}/y_test.pkl')
feature_names = joblib.load(f'{processed_dir}/feature_names.pkl')

print("="*60)
print("DATA LOADED")
print("="*60)
print(f"\nTest set: {X_test.shape[0]:,} samples × {X_test.shape[1]} features")
print(f"\nFeatures present: {len(feature_names)}")
print(f"Expected features: 24 (after data leakage removal)")

# Verify data leakage features have been removed
leakage_features = ['offer_completed', 'offer_viewed']
leakage_found = [f for f in leakage_features if f in feature_names]

if leakage_found:
    print(f"\n⚠️  WARNING: Leakage features still present: {leakage_found}")
    print("   These should have been removed in the modeling notebook!")
else:
    print(f"\n✓ Data leakage features removed successfully")

print(f"\nTarget distribution in test set:")
print(y_test.value_counts(normalize=True).round(3))

In [ ]:
rf_model = joblib.load(f'{models_dir}/random_forest.pkl')

print("="*60)
print("MODEL LOADED")
print("="*60)
print(f"Model type: Random Forest Classifier")
print(f"Expected features: {rf_model.n_features_in_}")
print(f"Test set features: {X_test.shape[1]}")

# Check feature alignment
if X_test.shape[1] != rf_model.n_features_in_:
    print(f"\n⚠️  FEATURE MISMATCH DETECTED!")
    print(f"   Model expects {rf_model.n_features_in_} features")
    print(f"   Test set has {X_test.shape[1]} features")
    print(f"\n   Aligning features...")
    
    # Align test set to match model expectations
    if X_test.shape[1] > rf_model.n_features_in_:
        # Remove extra features
        X_test = X_test.iloc[:, :rf_model.n_features_in_]
        feature_names = feature_names[:rf_model.n_features_in_]
        print(f"   ✓ Removed extra features. New shape: {X_test.shape}")
else:
    print(f"\n✓ Feature alignment verified")

# Generate predictions
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

print("\n" + "="*60)
print("MODEL PERFORMANCE ON TEST SET")
print("="*60)
print(f"  Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"  AUC-ROC:   {roc_auc_score(y_test, y_proba):.4f}")

print("\n" + "="*60)
print("EXPECTED PERFORMANCE (from 03_Modeling.ipynb)")
print("="*60)
print("  F1-Score:  0.8601")
print("  AUC-ROC:   0.9277")
print("\nNote: Values should match the modeling notebook results")
print("="*60)

## Protected Attributes Analysis

**Context from Previous Analyses:**

From **SHAP Analysis (05_SHAP.ipynb)**, we know the top predictive features:
1. **offer_type_discount** (21.4% importance) - Offer design feature
2. **duration** (14.2% importance) - Offer design feature  
3. **difficulty** (9.3% importance) - Offer design feature
4. **offer_viewed** (7.5% importance) - Behavioral feature
5. **gender_M** (5.9% importance) - **DEMOGRAPHIC FEATURE**
6. **income_bracket_encoded** (5.7% importance) - **DEMOGRAPHIC FEATURE**
7. **age** (5.3% importance) - **DEMOGRAPHIC FEATURE**

**Key Insight from SHAP:** Demographics account for 33.5% of model importance, with offer design at 52.3%. This means:
- The model DOES use demographic features significantly
- Gender, income, and age influence predictions
- **Potential for demographic bias exists**

From **Segmentation Analysis (06_Customer_Segmentation.ipynb)**, we know:
- **Cluster 0 (29%)**: 72.6% completion rate - High performers (long tenure, high engagement)
- **Cluster 1 (43%)**: 55.9% completion rate - Largest group (short tenure, perfect view rate)
- **Cluster 2 (12%)**: 15.7% completion rate - Data quality issue (missing demographics)
- **Cluster 3 (16%)**: 39.0% completion rate - Low engagement (zero view rate)

**Critical Question:** Are demographic groups distributed fairly across these segments? If not, certain groups may systematically receive worse predictions.

We'll analyze model performance across protected attributes from the original data to identify bias.

In [ ]:
df_original = pd.read_csv('../Cafe_Rewards_Offers/processed_data_for_classification.csv')

print(f"Original dataset loaded: {df_original.shape[0]:,} rows × {df_original.shape[1]} columns")
print(f"\nColumns available for fairness analysis:")
protected_cols = ['gender', 'age', 'income', 'age_group', 'income_bracket', 'tenure_group']
for col in protected_cols:
    if col in df_original.columns:
        unique_vals = df_original[col].unique()
        print(f"  - {col}: {len(unique_vals)} unique values")

In [ ]:
df_test_fairness = X_test.copy()
df_test_fairness['target'] = y_test.values
df_test_fairness['prediction'] = y_pred
df_test_fairness['prediction_proba'] = y_proba

df_original_test = df_original.iloc[X_test.index].copy()

for col in ['gender', 'age_group', 'income_bracket', 'tenure_group']:
    if col in df_original_test.columns:
        df_test_fairness[col] = df_original_test[col].values

print(f"Fairness analysis dataframe created: {df_test_fairness.shape}")
print(f"\nProtected attributes added: {['gender', 'age_group', 'income_bracket', 'tenure_group']}")

## Fairness Metrics Functions

In [ ]:
def calculate_group_metrics(y_true, y_pred, y_proba, group_mask):
    """Calculate classification metrics for a specific subgroup."""
    if sum(group_mask) < 10:
        return None
    
    y_true_g = y_true[group_mask]
    y_pred_g = y_pred[group_mask]
    y_proba_g = y_proba[group_mask]
    
    cm = confusion_matrix(y_true_g, y_pred_g)
    
    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    
    metrics = {
        'count': sum(group_mask),
        'positive_rate': y_pred_g.mean(),
        'accuracy': accuracy_score(y_true_g, y_pred_g),
        'precision': precision_score(y_true_g, y_pred_g, zero_division=0),
        'recall': recall_score(y_true_g, y_pred_g, zero_division=0),
        'f1': f1_score(y_true_g, y_pred_g, zero_division=0),
        'tpr': recall_score(y_true_g, y_pred_g, zero_division=0),
        'tnr': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'fpr': fp / (fp + tn) if (fp + tn) > 0 else 0,
        'fnr': fn / (fn + tp) if (fn + tp) > 0 else 0,
        'auc': roc_auc_score(y_true_g, y_proba_g) if len(np.unique(y_true_g)) > 1 else np.nan
    }
    
    return metrics


def analyze_fairness_by_attribute(df, attribute, y_true_col='target', 
                                  y_pred_col='prediction', y_proba_col='prediction_proba'):
    """Analyze fairness metrics across all values of a protected attribute."""
    
    results = []
    overall_metrics = calculate_group_metrics(
        df[y_true_col].values, 
        df[y_pred_col].values, 
        df[y_proba_col].values,
        np.ones(len(df), dtype=bool)
    )
    
    for value in df[attribute].unique():
        if pd.isna(value):
            continue
        
        mask = df[attribute] == value
        group_metrics = calculate_group_metrics(
            df[y_true_col].values, 
            df[y_pred_col].values, 
            df[y_proba_col].values,
            mask
        )
        
        if group_metrics:
            group_metrics['attribute'] = attribute
            group_metrics['value'] = value
            
            for metric in ['accuracy', 'precision', 'recall', 'f1', 'positive_rate', 'tpr', 'fpr']:
                if overall_metrics[metric] > 0:
                    diff = group_metrics[metric] - overall_metrics[metric]
                    group_metrics[f'{metric}_diff'] = diff
                    if overall_metrics[metric] > 0:
                        group_metrics[f'{metric}_pct_diff'] = (diff / overall_metrics[metric]) * 100
            
            results.append(group_metrics)
    
    return pd.DataFrame(results)


def calculate_disparate_impact(df, attribute, y_pred_col='prediction', reference_value=None):
    """Calculate disparate impact ratio for a protected attribute."""
    
    positive_rates = df.groupby(attribute)[y_pred_col].mean()
    
    if reference_value is None:
        reference_value = positive_rates.idxmax()
    
    reference_rate = positive_rates[reference_value]
    
    di_results = []
    for value, rate in positive_rates.items():
        if reference_rate > 0:
            di = rate / reference_rate
        else:
            di = np.nan
        
        di_results.append({
            'attribute': attribute,
            'value': value,
            'positive_rate': rate,
            'reference': reference_value,
            'reference_rate': reference_rate,
            'disparate_impact': di,
            'is_fair': 0.8 <= di <= 1.25
        })
    
    return pd.DataFrame(di_results)


def plot_fairness_comparison(metrics_df, attribute, metric_cols=['accuracy', 'precision', 'recall', 'f1']):
    """Plot fairness metrics comparison across groups."""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Fairness Analysis by {attribute}', fontsize=16, fontweight='bold')
    
    metrics_df = metrics_df.sort_values('value')
    values = metrics_df['value'].values
    
    for idx, metric in enumerate(metric_cols):
        ax = axes[idx // 2, idx % 2]
        
        bars = ax.bar(values, metrics_df[metric].values, alpha=0.7, edgecolor='black')
        
        overall_mean = metrics_df[f'{metric}_diff'].mean() + metrics_df[metric].mean()
        ax.axhline(y=overall_mean, color='red', linestyle='--', linewidth=2, 
                  label=f'Overall Mean: {overall_mean:.3f}')
        
        for bar, val in zip(bars, metrics_df[metric].values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{val:.3f}',
                   ha='center', va='bottom', fontsize=9)
        
        ax.set_xlabel(attribute)
        ax.set_ylabel(metric.replace('_', ' ').title())
        ax.set_title(f'{metric.replace("_", " ").title()} by {attribute}')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()


def plot_positive_rates(metrics_df, attribute):
    """Plot positive prediction rates across groups (Demographic Parity)."""
    
    plt.figure(figsize=(12, 6))
    
    metrics_df = metrics_df.sort_values('positive_rate')
    
    colors = ['green' if 0.8 <= (rate / metrics_df['positive_rate'].max()) <= 1.25 
              else 'orange' for rate in metrics_df['positive_rate']]
    
    bars = plt.bar(metrics_df['value'], metrics_df['positive_rate'], 
                    color=colors, alpha=0.7, edgecolor='black')
    
    plt.axhline(y=metrics_df['positive_rate'].mean(), color='red', 
                linestyle='--', linewidth=2, 
                label=f'Overall Mean: {metrics_df["positive_rate"].mean():.3f}')
    
    for bar, val in zip(bars, metrics_df['positive_rate'].values):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}',
                ha='center', va='bottom', fontsize=10)
    
    plt.xlabel(attribute)
    plt.ylabel('Positive Prediction Rate')
    plt.title(f'Demographic Parity - Positive Rate by {attribute}', 
              fontweight='bold')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

print("Fairness metrics functions defined! ✓")

## 1. Gender-based Fairness Analysis

In [ ]:
print("="*70)
print("GENDER DISTRIBUTION")
print("="*70)

gender_dist = df_test_fairness['gender'].value_counts(normalize=True).sort_index()
print("\nTest set distribution:")
for gender, pct in gender_dist.items():
    count = (df_test_fairness['gender'] == gender).sum()
    print(f"  {gender}: {count:5,} ({pct*100:5.2f}%)")

print("\nTarget completion rate by gender:")
for gender in df_test_fairness['gender'].unique():
    if pd.notna(gender):
        subset = df_test_fairness[df_test_fairness['gender'] == gender]
        completion_rate = subset['target'].mean()
        print(f"  {gender}: {completion_rate:.3f}")

In [ ]:
gender_fairness = analyze_fairness_by_attribute(df_test_fairness, 'gender')

print("="*70)
print("GENDER FAIRNESS METRICS")
print("="*70)

display_cols = ['value', 'count', 'accuracy', 'precision', 'recall', 'f1', 
                'positive_rate', 'tpr', 'fpr', 'auc']
print(gender_fairness[display_cols].round(4).to_string(index=False))

print("\n" + "="*70)
print("DISPARITIES FROM OVERALL")
print("="*70)

diff_cols = ['value', 'accuracy_pct_diff', 'precision_pct_diff', 
             'recall_pct_diff', 'f1_pct_diff']
print(gender_fairness[diff_cols].round(2).to_string(index=False))

In [ ]:
gender_di = calculate_disparate_impact(df_test_fairness, 'gender')

print("="*70)
print("GENDER DISPARATE IMPACT ANALYSIS")
print("="*70)
print("\nDisparate Impact Ratio = (Group Positive Rate) / (Reference Group Positive Rate)")
print("Fair range: 0.8 ≤ DI ≤ 1.25 (80% rule)")
print("\n")

for _, row in gender_di.iterrows():
    status = "✓ FAIR" if row['is_fair'] else "⚠️  UNFAIR"
    print(f"{row['value']:10} | Rate: {row['positive_rate']:.4f} | DI: {row['disparate_impact']:.3f} | {status}")

In [ ]:
plot_fairness_comparison(gender_fairness, 'gender')
plot_positive_rates(gender_fairness, 'gender')

## 2. Age Group-based Fairness Analysis

In [ ]:
print("="*70)
print("AGE GROUP DISTRIBUTION")
print("="*70)

age_dist = df_test_fairness['age_group'].value_counts().sort_index()
print("\nTest set distribution:")
for age, count in age_dist.items():
    if pd.notna(age):
        pct = (count / len(df_test_fairness)) * 100
        print(f"  {age:10}: {count:6,} ({pct:5.2f}%)")

print("\nTarget completion rate by age group:")
for age in sorted(df_test_fairness['age_group'].unique()):
    if pd.notna(age):
        subset = df_test_fairness[df_test_fairness['age_group'] == age]
        completion_rate = subset['target'].mean()
        print(f"  {age:10}: {completion_rate:.3f}")

In [ ]:
age_fairness = analyze_fairness_by_attribute(df_test_fairness, 'age_group')

print("="*70)
print("AGE GROUP FAIRNESS METRICS")
print("="*70)

display_cols = ['value', 'count', 'accuracy', 'precision', 'recall', 'f1', 
                'positive_rate', 'tpr', 'fpr', 'auc']
print(age_fairness[display_cols].round(4).to_string(index=False))

print("\n" + "="*70)
print("DISPARITIES FROM OVERALL")
print("="*70)

diff_cols = ['value', 'accuracy_pct_diff', 'precision_pct_diff', 
             'recall_pct_diff', 'f1_pct_diff']
print(age_fairness[diff_cols].round(2).to_string(index=False))

In [ ]:
age_di = calculate_disparate_impact(df_test_fairness, 'age_group')

print("="*70)
print("AGE GROUP DISPARATE IMPACT ANALYSIS")
print("="*70)
print("\nDisparate Impact Ratio = (Group Positive Rate) / (Reference Group Positive Rate)")
print("Fair range: 0.8 ≤ DI ≤ 1.25 (80% rule)")
print("\n")

for _, row in age_di.iterrows():
    status = "✓ FAIR" if row['is_fair'] else "⚠️  UNFAIR"
    print(f"{row['value']:10} | Rate: {row['positive_rate']:.4f} | DI: {row['disparate_impact']:.3f} | {status}")

In [ ]:
plot_fairness_comparison(age_fairness, 'age_group')
plot_positive_rates(age_fairness, 'age_group')

## 3. Income Bracket-based Fairness Analysis

In [ ]:
print("="*70)
print("INCOME BRACKET DISTRIBUTION")
print("="*70)

income_dist = df_test_fairness['income_bracket'].value_counts().sort_index()
print("\nTest set distribution:")
for income, count in income_dist.items():
    if pd.notna(income):
        pct = (count / len(df_test_fairness)) * 100
        print(f"  {income:12}: {count:6,} ({pct:5.2f}%)")

print("\nTarget completion rate by income bracket:")
for income in sorted(df_test_fairness['income_bracket'].unique()):
    if pd.notna(income):
        subset = df_test_fairness[df_test_fairness['income_bracket'] == income]
        completion_rate = subset['target'].mean()
        print(f"  {income:12}: {completion_rate:.3f}")

In [ ]:
income_fairness = analyze_fairness_by_attribute(df_test_fairness, 'income_bracket')

print("="*70)
print("INCOME BRACKET FAIRNESS METRICS")
print("="*70)

display_cols = ['value', 'count', 'accuracy', 'precision', 'recall', 'f1', 
                'positive_rate', 'tpr', 'fpr', 'auc']
print(income_fairness[display_cols].round(4).to_string(index=False))

print("\n" + "="*70)
print("DISPARITIES FROM OVERALL")
print("="*70)

diff_cols = ['value', 'accuracy_pct_diff', 'precision_pct_diff', 
             'recall_pct_diff', 'f1_pct_diff']
print(income_fairness[diff_cols].round(2).to_string(index=False))

In [ ]:
income_di = calculate_disparate_impact(df_test_fairness, 'income_bracket')

print("="*70)
print("INCOME BRACKET DISPARATE IMPACT ANALYSIS")
print("="*70)
print("\nDisparate Impact Ratio = (Group Positive Rate) / (Reference Group Positive Rate)")
print("Fair range: 0.8 ≤ DI ≤ 1.25 (80% rule)")
print("\n")

for _, row in income_di.iterrows():
    status = "✓ FAIR" if row['is_fair'] else "⚠️  UNFAIR"
    print(f"{row['value']:12} | Rate: {row['positive_rate']:.4f} | DI: {row['disparate_impact']:.3f} | {status}")

In [ ]:
plot_fairness_comparison(income_fairness, 'income_bracket')
plot_positive_rates(income_fairness, 'income_bracket')

## 4. Tenure Group-based Fairness Analysis

In [ ]:
print("="*70)
print("TENURE GROUP DISTRIBUTION")
print("="*70)

tenure_dist = df_test_fairness['tenure_group'].value_counts()
print("\nTest set distribution:")
for tenure, count in tenure_dist.items():
    if pd.notna(tenure):
        pct = (count / len(df_test_fairness)) * 100
        print(f"  {tenure:15}: {count:6,} ({pct:5.2f}%)")

print("\nTarget completion rate by tenure group:")
for tenure in df_test_fairness['tenure_group'].unique():
    if pd.notna(tenure):
        subset = df_test_fairness[df_test_fairness['tenure_group'] == tenure]
        completion_rate = subset['target'].mean()
        print(f"  {tenure:15}: {completion_rate:.3f}")

In [ ]:
tenure_fairness = analyze_fairness_by_attribute(df_test_fairness, 'tenure_group')

print("="*70)
print("TENURE GROUP FAIRNESS METRICS")
print("="*70)

display_cols = ['value', 'count', 'accuracy', 'precision', 'recall', 'f1', 
                'positive_rate', 'tpr', 'fpr', 'auc']
print(tenure_fairness[display_cols].round(4).to_string(index=False))

print("\n" + "="*70)
print("DISPARITIES FROM OVERALL")
print("="*70)

diff_cols = ['value', 'accuracy_pct_diff', 'precision_pct_diff', 
             'recall_pct_diff', 'f1_pct_diff']
print(tenure_fairness[diff_cols].round(2).to_string(index=False))

In [ ]:
plot_fairness_comparison(tenure_fairness, 'tenure_group')
plot_positive_rates(tenure_fairness, 'tenure_group')

## 5. Confusion Matrices by Protected Groups

In [ ]:
def plot_group_confusion_matrix(y_true, y_pred, group_name, ax):
    cm = confusion_matrix(y_true, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax,
                xticklabels=['Not Completed', 'Completed'],
                yticklabels=['Not Completed', 'Completed'])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(group_name, fontweight='bold')


fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Confusion Matrices by Gender', fontsize=16, fontweight='bold')

gender_values = [g for g in df_test_fairness['gender'].unique() if pd.notna(g)]
axes_flat = axes.flatten()

for idx, gender in enumerate(gender_values):
    mask = df_test_fairness['gender'] == gender
    plot_group_confusion_matrix(
        df_test_fairness[mask]['target'],
        df_test_fairness[mask]['prediction'],
        f'Gender: {gender} (n={mask.sum():,})',
        axes_flat[idx]
    )

for idx in range(len(gender_values), len(axes_flat)):
    axes_flat[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Confusion Matrices by Age Group', fontsize=16, fontweight='bold')

age_values = sorted([a for a in df_test_fairness['age_group'].unique() if pd.notna(a)])
axes_flat = axes.flatten()

for idx, age in enumerate(age_values):
    mask = df_test_fairness['age_group'] == age
    plot_group_confusion_matrix(
        df_test_fairness[mask]['target'],
        df_test_fairness[mask]['prediction'],
        f'Age: {age} (n={mask.sum():,})',
        axes_flat[idx]
    )

for idx in range(len(age_values), len(axes_flat)):
    axes_flat[idx].axis('off')

plt.tight_layout()
plt.show()

## 6. ROC Curves by Protected Groups

In [ ]:
def plot_roc_by_group(df, attribute, title_suffix):
    plt.figure(figsize=(10, 8))
    
    values = sorted([v for v in df[attribute].unique() if pd.notna(v)])
    
    for value in values:
        mask = df[attribute] == value
        if sum(mask) < 10:
            continue
        
        y_true = df[mask]['target']
        y_proba = df[mask]['prediction_proba']
        
        if len(np.unique(y_true)) < 2:
            continue
        
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = roc_auc_score(y_true, y_proba)
        
        plt.plot(fpr, tpr, label=f'{value} (AUC = {auc_score:.3f})', linewidth=2)
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves by {title_suffix}', fontweight='bold')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_roc_by_group(df_test_fairness, 'gender', 'Gender')
plot_roc_by_group(df_test_fairness, 'age_group', 'Age Group')
plot_roc_by_group(df_test_fairness, 'income_bracket', 'Income Bracket')

## 7. Intersectional Fairness Analysis

Analyze fairness across intersections of protected attributes (e.g., Gender × Age Group).

In [ ]:
print("="*70)
print("INTERSECTIONAL FAIRNESS: GENDER × AGE GROUP")
print("="*70)

intersectional_results = []

for gender in ['M', 'F']:
    for age in sorted([a for a in df_test_fairness['age_group'].unique() if pd.notna(a)]):
        mask = (df_test_fairness['gender'] == gender) & (df_test_fairness['age_group'] == age)
        
        if sum(mask) < 10:
            continue
        
        y_true = df_test_fairness[mask]['target']
        y_pred = df_test_fairness[mask]['prediction']
        
        metrics = {
            'gender': gender,
            'age_group': age,
            'count': sum(mask),
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'positive_rate': y_pred.mean()
        }
        
        intersectional_results.append(metrics)

intersectional_df = pd.DataFrame(intersectional_results)

print("\nIntersectional Fairness Metrics:")
print(intersectional_df.round(4).to_string(index=False))

print("\n" + "="*70)
print("MAXIMUM DISPARITY IN INTERSECTIONAL GROUPS")
print("="*70)

for metric in ['accuracy', 'precision', 'recall', 'f1', 'positive_rate']:
    if len(intersectional_df) > 0:
        max_val = intersectional_df[metric].max()
        min_val = intersectional_df[metric].min()
        disparity = max_val - min_val
        print(f"{metric:15}: Max={max_val:.3f}, Min={min_val:.3f}, Disparity={disparity:.3f}")

In [ ]:
pivot_accuracy = intersectional_df.pivot(index='age_group', columns='gender', values='accuracy')
pivot_recall = intersectional_df.pivot(index='age_group', columns='gender', values='recall')
pivot_f1 = intersectional_df.pivot(index='age_group', columns='gender', values='f1')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.heatmap(pivot_accuracy, annot=True, fmt='.3f', cmap='RdYlGn', cbar_kws={'label': 'Accuracy'}, ax=axes[0])
axes[0].set_title('Accuracy by Gender × Age', fontweight='bold')

sns.heatmap(pivot_recall, annot=True, fmt='.3f', cmap='RdYlGn', cbar_kws={'label': 'Recall'}, ax=axes[1])
axes[1].set_title('Recall by Gender × Age', fontweight='bold')

sns.heatmap(pivot_f1, annot=True, fmt='.3f', cmap='RdYlGn', cbar_kws={'label': 'F1-Score'}, ax=axes[2])
axes[2].set_title('F1-Score by Gender × Age', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("INTERPRETING INTERSECTIONAL FAIRNESS")
print("="*60)
print("\nKey Patterns to Look For:")
print("  • Dark green cells (high F1): Well-served demographic groups")
print("  • Red cells (low F1): Under-served groups needing intervention")
print("  • Consistent rows: Age matters more than gender for this row")
print("  • Consistent columns: Gender matters more than age for this column")
print("\nFairness Concern: If low F1 is concentrated in specific intersections")
print("(e.g., 'Young × Low Income'), those groups face compounded disadvantage.")

## 7.2 Linking Demographics to Customer Segments

**Critical Analysis:** From the segmentation notebook (06_Customer_Segmentation.ipynb), we identified customer segments with vastly different completion rates:
- Cluster 0: 72.6% completion
- Cluster 1: 55.9% completion  
- Cluster 2: 15.7% completion ⚠️
- Cluster 3: 39.0% completion

**Key Question:** Are certain demographic groups overrepresented in low-performing segments?

If yes, this creates **systemic bias** where demographic characteristics indirectly determine segment assignment, which determines offer success.

In [ ]:
# Load segmentation results
segmentation_dir = '../Cafe_Rewards_Offers/segmentation'

try:
    customers_with_clusters = pd.read_csv(f'{segmentation_dir}/customers_with_clusters.csv')
    print("="*70)
    print("SEGMENTATION DATA LOADED")
    print("="*70)
    print(f"Total customers with cluster assignments: {len(customers_with_clusters):,}")
    print(f"\nCluster distribution:")
    print(customers_with_clusters['cluster'].value_counts().sort_index())
    
    # Check if we can merge with test set
    print(f"\nTest set size: {len(df_test_fairness):,}")
    print(f"Cluster data size: {len(customers_with_clusters):,}")
    
    # Add cluster assignments to test fairness dataframe if indices match
    if len(customers_with_clusters) == len(df_original):
        # Extract cluster for test set indices
        df_test_fairness['cluster'] = customers_with_clusters.iloc[df_test_fairness.index]['cluster'].values
        
        print("\n✓ Successfully added cluster assignments to test set")
        print(f"\nTest set cluster distribution:")
        print(df_test_fairness['cluster'].value_counts().sort_index())
        
        # Analyze demographics by cluster
        print("\n" + "="*70)
        print("DEMOGRAPHIC COMPOSITION BY CUSTOMER SEGMENT")
        print("="*70)
        
        for cluster in sorted(df_test_fairness['cluster'].unique()):
            if pd.isna(cluster):
                continue
            
            cluster_data = df_test_fairness[df_test_fairness['cluster'] == cluster]
            
            print(f"\n{'='*70}")
            print(f"CLUSTER {int(cluster)}")
            print(f"{'='*70}")
            print(f"Size: {len(cluster_data):,} customers ({len(cluster_data)/len(df_test_fairness)*100:.1f}% of test set)")
            print(f"Completion rate: {cluster_data['target'].mean():.1%}")
            
            # Gender distribution
            print(f"\nGender distribution:")
            for gender in ['M', 'F', 'O', 'Missing']:
                if 'gender' in cluster_data.columns:
                    pct = (cluster_data['gender'] == gender).sum() / len(cluster_data) * 100
                    print(f"  {gender:8}: {pct:5.1f}%")
            
            # Age distribution
            print(f"\nAge group distribution:")
            if 'age_group' in cluster_data.columns:
                age_dist = cluster_data['age_group'].value_counts(normalize=True).sort_index() * 100
                for age, pct in age_dist.items():
                    if pd.notna(age):
                        print(f"  {age:10}: {pct:5.1f}%")
            
            # Income distribution
            print(f"\nIncome bracket distribution:")
            if 'income_bracket' in cluster_data.columns:
                income_dist = cluster_data['income_bracket'].value_counts(normalize=True).sort_index() * 100
                for income, pct in income_dist.items():
                    if pd.notna(income):
                        print(f"  {income:12}: {pct:5.1f}%")
        
        print("\n" + "="*70)
        print("KEY INSIGHTS")
        print("="*70)
        print("\n1. Check if Cluster 2 (lowest completion 15.7%) has:")
        print("   - Higher % of missing demographics")
        print("   - Specific age/income groups overrepresented")
        print("\n2. Check if Cluster 0 (highest completion 72.6%) has:")
        print("   - Specific demographic advantages")
        print("   - Exclusion of certain groups")
        print("\n3. Fairness concern if:")
        print("   - Protected groups are trapped in low-performing clusters")
        print("   - Demographic features drive cluster assignment")
        
    else:
        print("\n⚠️  Cannot merge: Cluster data size doesn't match original data")
        print("   This analysis requires cluster assignments for the exact test set indices")
        
except FileNotFoundError:
    print("⚠️  Segmentation results not found")
    print(f"   Expected location: {segmentation_dir}/customers_with_clusters.csv")
    print("   Run 06_Customer_Segmentation.ipynb first")

## 8. Summary and Recommendations

### Integration with Previous Analyses

This fairness analysis completes our comprehensive evaluation of the offer completion prediction system. Let's synthesize findings across all notebooks:

#### From Modeling (03_Modeling.ipynb):
- **Best Model**: Random Forest (F1: 0.8601, AUC: 0.9277)
- **Data Leakage Removed**: 24 features (removed offer_completed, offer_viewed)
- **Feature Count**: 24 clean features for real-time prediction
- **Class Balance**: 53.4% completed, 46.6% not completed (well-balanced)

#### From PCA (04_PCA.ipynb):
- **Dimensionality Reduction**: 8 components capture 90% variance
- **Performance Trade-off**: Only 1% F1 drop (0.8601 → 0.8563) with 68% fewer features
- **Feature Concentration**: Top 10 features explain 83% of importance
- **Recommendation**: Use 8-component PCA for production efficiency

#### From SHAP (05_SHAP.ipynb):
- **Top 3 Features Account for 45% of Predictions**:
  1. offer_type_discount: 21.4%
  2. duration: 14.2%
  3. difficulty: 9.3%
- **Feature Categories**:
  - Offer Attributes: 52.3% (LARGEST)
  - Demographics: 33.5% (gender, income, age)
  - Behavioral: 7.5% (offer_viewed)
  - Channels: 4.4%
- **Key Demographic Features**:
  - gender_M: 5.9% importance (rank #5)
  - income_bracket_encoded: 5.7% (rank #7)
  - age: 5.3% (rank #8)

**FAIRNESS IMPLICATION**: Demographics account for 1/3 of prediction power. If certain groups systematically have different demographics, the model will treat them differently.

#### From Segmentation (06_Customer_Segmentation.ipynb):
- **5 Customer Segments Identified**:
  - Cluster 0 (29%): 72.6% completion - Long-tenure, high engagement
  - Cluster 1 (43%): 55.9% completion - Short-tenure, perfect view rate
  - Cluster 2 (12%): 15.7% completion - **Missing demographics** (data quality issue)
  - Cluster 3 (16%): 39.0% completion - Low engagement, zero view rate
  - Cluster 4 (1%): Small outlier segment

**FAIRNESS IMPLICATION**: Cluster 2 (12% of customers) has missing demographics and performs **5x worse** than the best segment. If certain demographic groups are overrepresented in this cluster, they experience systematic disadvantage.

**Potential Business Impact**:
- If segmentation strategies are implemented: +12,568 additional completions (27.3% increase)
- Cluster 1 improvement potential: +6,169 conversions
- Cluster 2 data fix potential: +4,004 conversions
- Cluster 3 re-engagement: +2,395 conversions

In [ ]:
def generate_fairness_summary(gender_fairness, age_fairness, income_fairness, tenure_fairness):
    """Generate a comprehensive fairness summary."""
    
    summary = {}
    
    fairness_dfs = {
        'gender': gender_fairness,
        'age_group': age_fairness,
        'income_bracket': income_fairness,
        'tenure_group': tenure_fairness
    }
    
    for attr_name, df in fairness_dfs.items():
        if df is None or len(df) == 0:
            continue
        
        metrics_to_check = ['accuracy', 'precision', 'recall', 'f1', 'positive_rate']
        
        summary[attr_name] = {}
        
        for metric in metrics_to_check:
            if f'{metric}_pct_diff' in df.columns:
                max_diff = df[f'{metric}_pct_diff'].abs().max()
                min_diff = df[f'{metric}_pct_diff'].abs().min()
                
                if max_diff > 10:
                    risk_level = "HIGH"
                elif max_diff > 5:
                    risk_level = "MEDIUM"
                else:
                    risk_level = "LOW"
                
                summary[attr_name][metric] = {
                    'max_disparity_pct': round(max_diff, 2),
                    'risk_level': risk_level
                }
    
    return summary


fairness_summary = generate_fairness_summary(
    gender_fairness, age_fairness, income_fairness, tenure_fairness
)

print("="*70)
print("FAIRNESS ANALYSIS SUMMARY")
print("="*70)

for attr_name, metrics in fairness_summary.items():
    print(f"\n{attr_name.upper()}:")
    for metric_name, values in metrics.items():
        risk_emoji = {
            'HIGH': '🔴',
            'MEDIUM': '🟡',
            'LOW': '🟢'
        }[values['risk_level']]
        print(f"  {metric_name:15}: {values['max_disparity_pct']:>6.2f}% disparity - {risk_emoji} {values['risk_level']}")

In [ ]:
print("="*70)
print("ACTIONABLE RECOMMENDATIONS")
print("="*70)
print("\nBased on integrated findings from all analyses:")

recommendations = [
    {
        'category': '1. IMMEDIATE: Address Data Quality Bias (Week 1)',
        'priority': 'CRITICAL',
        'items': [
            "From Segmentation: 12% of customers (Cluster 2) have missing demographics",
            "These customers have 15.7% completion vs 72.6% for best segment (4.6x worse)",
            "If certain demographic groups are less likely to provide data, this creates systematic bias",
            "",
            "Actions:",
            "  • Audit: Which demographics are overrepresented in 'missing data' group?",
            "  • Fix onboarding: Make key demographic fields required (age, income, gender)",
            "  • Incentivize: Offer free drink/points for profile completion",
            "  • Target: Launch campaign to existing customers with incomplete profiles",
            "",
            "Expected Impact: +4,004 conversions if Cluster 2 is brought to average performance"
        ]
    },
    {
        'category': '2. SHORT-TERM: Fairness Monitoring Dashboard (Month 1)',
        'priority': 'HIGH',
        'items': [
            "From SHAP: Demographics drive 33.5% of predictions (gender 5.9%, income 5.7%, age 5.3%)",
            "From Fairness Analysis: Performance varies by protected attribute (computed above)",
            "",
            "Actions:",
            "  • Set up automated fairness metric tracking by:",
            "    - Gender (M, F, O, Missing)",
            "    - Age group (18-30, 31-45, 46-60, 61+)",
            "    - Income bracket (Low, Medium, High, Very High)",
            "    - Customer segment (Clusters 0-3)",
            "  • Alert when disparities exceed thresholds:",
            "    - Accuracy disparity > 10%",
            "    - Disparate impact < 0.8 or > 1.25 (80% rule violation)",
            "    - F1-score gap > 0.15 between best/worst group",
            "  • Weekly dashboard review with stakeholders",
            "",
            "Tools: MLflow, Evidently AI, or custom dashboard"
        ]
    },
    {
        'category': '3. SHORT-TERM: Optimize Offer Design for Fairness (Month 1-2)',
        'priority': 'HIGH',
        'items': [
            "From SHAP: Offer design drives 52.3% of predictions (discount 21%, duration 14%, difficulty 9%)",
            "GOOD NEWS: Offer features are controllable, unlike demographics",
            "",
            "Actions:",
            "  • Test if fairness improves when using offer-based features ONLY:",
            "    - Retrain model WITHOUT demographic features",
            "    - Compare: Does accuracy drop significantly?",
            "    - Compare: Do fairness metrics improve?",
            "  • Personalize offers by behavior, not demographics:",
            "    - Use: offer_viewed, tenure, past behavior",
            "    - Avoid: gender, age, income as primary drivers",
            "  • Create 'universal offers' that work across all demographics:",
            "    - Short duration (5-7 days)",
            "    - Low difficulty (easy thresholds)",
            "    - Discount type (highest completion: 54.9% vs 28.4% for others)",
            "",
            "Expected Impact: Reduce demographic bias while maintaining performance"
        ]
    },
    {
        'category': '4. MEDIUM-TERM: Segment-Specific Strategies (Month 2-3)',
        'priority': 'MEDIUM',
        'items': [
            "From Segmentation: Clear performance tiers exist (15.7% to 72.6% completion)",
            "From Fairness: Check if certain demographics are trapped in low-performing segments",
            "",
            "Actions for each segment:",
            "",
            "CLUSTER 0 (72.6% completion, 29% of customers) - HIGH PERFORMERS:",
            "  • Maintain satisfaction: VIP treatment, exclusive offers",
            "  • Use as fairness baseline: All groups should perform like this",
            "  • Analyze: What demographics dominate this cluster?",
            "",
            "CLUSTER 1 (55.9% completion, 43% of customers) - LARGEST OPPORTUNITY:",
            "  • Educational content: 'How to maximize rewards'",
            "  • Progressive difficulty: Start easy, increase over time",
            "  • Target improvement: +6,169 conversions if improved to 72.6%",
            "",
            "CLUSTER 2 (15.7% completion, 12% of customers) - DATA QUALITY CRISIS:",
            "  • Fix missing demographics (see Recommendation #1)",
            "  • Use behavioral features until demographics collected",
            "  • Target improvement: +4,004 conversions",
            "",
            "CLUSTER 3 (39.0% completion, 16% of customers) - RE-ENGAGEMENT:",
            "  • Zero view rate but 39% completion (unusual!)",
            "  • Push notifications, email campaigns, in-app banners",
            "  • Target improvement: +2,395 conversions",
            "",
            "Total Opportunity: +12,568 conversions (27.3% increase)"
        ]
    },
    {
        'category': '5. MEDIUM-TERM: Fairness-Aware Modeling (Month 2-3)',
        'priority': 'MEDIUM',
        'items': [
            "IF fairness violations detected above (disparate impact < 0.8 or > 1.25):",
            "",
            "Option A: Reweighting",
            "  • Increase sample weights for under-predicted groups during training",
            "  • Decrease weights for over-predicted groups",
            "  • Trade-off: May reduce overall accuracy by 1-3%",
            "",
            "Option B: Adversarial Debiasing",
            "  • Train model to predict offers well BUT hide demographic info",
            "  • Use adversarial network that tries to predict demographics from predictions",
            "  • Force model to be 'blind' to protected attributes",
            "",
            "Option C: Post-Processing Calibration",
            "  • Train separate threshold for each demographic group",
            "  • Adjust prediction thresholds to equalize TPR or precision",
            "  • Easiest to implement, doesn't require retraining",
            "",
            "Option D: Remove Demographic Features Entirely",
            "  • Retrain using only: offer features, behavioral features, tenure",
            "  • Test: Does PCA with 8 components maintain fairness?",
            "  • From PCA: Only 1% F1 drop with 68% fewer features",
            "",
            "Recommended: Start with Option D (simplest), then try Option C if needed"
        ]
    },
    {
        'category': '6. LONG-TERM: Governance & Documentation (Ongoing)',
        'priority': 'ONGOING',
        'items': [
            "Documentation:",
            "  • Create Model Card documenting:",
            "    - Intended use (offer completion prediction for marketing)",
            "    - Training data demographics (% by gender, age, income)",
            "    - Known biases and limitations",
            "    - Fairness metrics and thresholds",
            "    - Mitigation strategies implemented",
            "  • Document decisions:",
            "    - Why use demographics vs behavioral-only?",
            "    - What fairness-accuracy trade-offs are acceptable?",
            "    - Who approves fairness threshold changes?",
            "",
            "Governance:",
            "  • Quarterly fairness audits (minimum)",
            "  • Annual deep-dive analysis like this notebook",
            "  • Stakeholder review board for fairness decisions",
            "  • Regulatory compliance check (EEOC, GDPR, CCPA)",
            "",
            "Continuous Improvement:",
            "  • A/B test fairness interventions",
            "  • Track customer satisfaction by demographic group",
            "  • Monitor complaints/feedback for bias concerns",
            "  • Update model when fairness metrics degrade"
        ]
    }
]

for rec in recommendations:
    print(f"\n{'='*70}")
    print(f"{rec['category']}")
    if 'priority' in rec:
        priority_emoji = {
            'CRITICAL': '🔴',
            'HIGH': '🟠',
            'MEDIUM': '🟡',
            'ONGOING': '🔵'
        }.get(rec['priority'], '')
        print(f"{priority_emoji} Priority: {rec['priority']}")
    print(f"{'='*70}")
    for item in rec['items']:
        if item == "":
            print()
        elif item.startswith("  "):
            print(item)
        else:
            print(f"{item}")

print("\n" + "="*70)
print("SUMMARY: PRIORITIZED ACTION PLAN")
print("="*70)
print("\n🔴 WEEK 1 (Critical):")
print("  → Fix data collection for Cluster 2 (missing demographics)")
print("  → Review fairness metrics computed above")
print("  → Identify groups with >10% disparity")
print("\n🟠 MONTH 1 (High):")
print("  → Set up fairness monitoring dashboard")
print("  → Test model WITHOUT demographic features")
print("  → Optimize offer design (short, easy, discount)")
print("\n🟡 MONTHS 2-3 (Medium):")
print("  → Implement segment-specific strategies")
print("  → Apply fairness-aware modeling if needed")
print("  → A/B test interventions")
print("\n🔵 ONGOING:")
print("  → Quarterly fairness audits")
print("  → Model cards and documentation")
print("  → Stakeholder reviews")
print("\nExpected Total Impact: +12,568 conversions (27.3% increase) with improved fairness")

In [ ]:
os.makedirs('../Cafe_Rewards_Offers/fairness_analysis', exist_ok=True)

gender_fairness.to_csv('../Cafe_Rewards_Offers/fairness_analysis/gender_fairness.csv', index=False)
age_fairness.to_csv('../Cafe_Rewards_Offers/fairness_analysis/age_fairness.csv', index=False)
income_fairness.to_csv('../Cafe_Rewards_Offers/fairness_analysis/income_fairness.csv', index=False)
tenure_fairness.to_csv('../Cafe_Rewards_Offers/fairness_analysis/tenure_fairness.csv', index=False)
intersectional_df.to_csv('../Cafe_Rewards_Offers/fairness_analysis/intersectional_fairness.csv', index=False)

print("="*70)
print("✓ FAIRNESS ANALYSIS RESULTS SAVED")
print("="*70)
print("\nSaved files:")
print("  - gender_fairness.csv")
print("  - age_fairness.csv")
print("  - income_fairness.csv")
print("  - tenure_fairness.csv")
print("  - intersectional_fairness.csv")
print("\nLocation: ../Cafe_Rewards_Offers/fairness_analysis/")

## Conclusion

This notebook provides a comprehensive bias and fairness analysis of the offer completion prediction model, integrated with findings from all previous analyses.

### What Was Analyzed:
1. **Protected Attributes**: Gender, Age Group, Income Bracket, Tenure Group
2. **Fairness Metrics**: Accuracy, Precision, Recall, F1-Score, Positive Rate, TPR, FPR, AUC
3. **Bias Types**: Demographic parity, equal opportunity, predictive parity
4. **Intersectional Analysis**: Combinations of protected attributes

### Fairness Frameworks Applied:
- **Demographic Parity (80% Rule)**: Disparate impact between 0.8-1.25
- **Equal Opportunity**: Similar true positive rates across groups
- **Predictive Parity**: Similar precision across groups
- **Individual Fairness**: Similar predictions for similar individuals

### Key Findings from Integrated Analysis:

#### 1. Model Design Creates Inherent Bias Risk
From SHAP analysis, we know:
- **Demographics drive 33.5% of predictions** (gender 5.9%, income 5.7%, age 5.3%)
- **Offer design drives 52.3%** (type, duration, difficulty)
- This means demographic features ARE used significantly in predictions
- **Risk**: Groups with different demographic distributions will receive different prediction rates

#### 2. Segmentation Reveals Systematic Disadvantage
From customer segmentation:
- **Cluster 2 (12% of customers)** has missing demographics and **15.7% completion rate**
- Best-performing Cluster 0 has **72.6% completion rate** (4.6x better)
- If certain demographic groups are overrepresented in low-performing clusters, they face systematic disadvantage
- **Critical**: The fairness analysis in this notebook should reveal if specific groups are concentrated in poor-performing clusters

#### 3. Data Quality Drives Fairness
- Segmentation identified 9,963 customers (12%) with missing demographics
- These customers perform **5x worse** than high performers
- If demographic data collection is biased (e.g., certain groups less likely to provide info), this creates unfairness
- **Action**: Audit data collection process for demographic bias

#### 4. Feature Importance Reveals Bias Mechanisms
SHAP showed offer design matters more than demographics (52% vs 34%), which is GOOD for fairness:
- Offer design features (discount, duration, difficulty) are **controllable**
- Demographic features are **immutable** (can't change age, income overnight)
- **Opportunity**: Improve fairness by optimizing offer design rather than relying on demographics

### Critical Questions Answered by This Analysis:

The cells above should have revealed:

1. **Are certain demographic groups systematically under-predicted or over-predicted?**
   - Check gender, age, and income fairness metrics
   - Disparate impact ratios should be between 0.8-1.25
   
2. **Do high-performing customer segments over-represent certain demographics?**
   - Cross-reference segment membership with protected attributes
   - If Cluster 0 (72.6% completion) is predominantly high-income males, this creates bias
   
3. **Are fairness violations due to model bias or real behavioral differences?**
   - If SHAP shows demographics are important AND fairness metrics show disparities, likely model bias
   - If demographics aren't important but disparities exist, likely real behavioral differences
   
4. **Which groups need targeted interventions?**
   - Groups with high FPR: receiving offers they won't complete (wasted marketing spend)
   - Groups with high FNR: missing offers they would complete (lost revenue)
   - Groups in low-performing segments: need re-engagement strategies

### Next Steps:

#### Immediate Actions (Week 1-2):
1. **Review the fairness metrics computed above**
   - Identify any group with >10% disparity in accuracy, precision, or recall
   - Flag any disparate impact violations (ratio < 0.8 or > 1.25)
   
2. **Cross-reference with segmentation results**
   - Compute demographic distribution across customer segments
   - Identify if certain groups are trapped in low-performing segments
   
3. **Audit data collection for demographic bias**
   - Why do 12% of customers have missing demographics?
   - Are certain groups less likely to provide this information?

#### Short-term (Month 1-2):
1. **If bias is detected:**
   - Consider fairness-aware algorithms (e.g., reweighting, adversarial debiasing)
   - Apply post-processing calibration to equalize predictions across groups
   - Test model performance with/without demographic features
   
2. **If bias is minimal:**
   - Document findings and create model cards
   - Set up ongoing fairness monitoring in production
   - Establish thresholds for acceptable disparities

#### Long-term (Quarter 1-2):
1. **Implement fairness monitoring dashboard**
   - Track fairness metrics in real-time by protected attribute
   - Alert when disparities exceed thresholds
   - Regular audits (quarterly minimum)
   
2. **Balance business objectives with fairness**
   - Define acceptable fairness-accuracy trade-offs
   - Document decisions and rationale
   - Engage stakeholders in fairness discussions
   
3. **Improve data quality**
   - Fix onboarding to collect complete demographics
   - Offer incentives for profile completion
   - Reduce Cluster 2 (missing demographics) to <5% of customers

### Key Takeaways:

✅ **The model DOES use demographics** (33.5% of prediction importance)
✅ **Customer segments show 4.6x performance gap** (15.7% to 72.6%)
✅ **12% of customers have missing demographics** and perform poorly
✅ **The fairness analysis above reveals if bias exists** across protected groups
⚠️ **Action required**: Review all fairness metrics and implement mitigation strategies

**Remember**: Fairness is context-dependent and requires ongoing monitoring. This analysis provides a snapshot; production deployment requires continuous evaluation.